In [9]:
import requests
import pandas as pd 


In [25]:
def getdata(tableName,tags, start_time, end_time):
    url = 'http://10.79.58.13/kawai/api/v1/readdata'
    try:
        # API request
        response = requests.post(url,
            json={
                "tableName": tableName,
                "sensorids": tags,
                "starttime": start_time,
                "endtime"  : end_time
            },
            timeout=10  # Timeout to prevent hanging
        )
        
        # Check HTTP status
        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code} from the server.")
            return pd.DataFrame()

        # Parse JSON response
        response_data = response.json()
        
        # Check if data exists
        if not response_data or "data" not in response_data or not response_data["data"]:
            print("Warning: No data received for the specified inputs.")
            return pd.DataFrame()

        # Convert to DataFrame
        df = pd.DataFrame(response_data["data"])
        if df.empty:
            print("Warning: Data is empty after conversion to DataFrame.")
            return pd.DataFrame()

        # Ensure 'time' column is datetime and pivot the DataFrame
        df["time"] = pd.to_datetime(df["time"], errors='coerce')
        pivot_df = df.pivot(index='time', columns='sensorid', values='measurement').reset_index()

        # Clean column names
        pivot_df.columns.name = None
        pivot_df = pivot_df.rename_axis(None, axis=1)

        return pivot_df

    except requests.exceptions.RequestException as e:
        print(f"Error: An exception occurred while making the API request - {e}")
        return pd.DataFrame()
    except ValueError as e:
        print(f"Error: Failed to process the JSON response - {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error: An unexpected error occurred - {e}")
        return pd.DataFrame()
        
        
        
df = getdata("sensordataKawai",["KAWAI_U2_2ND_STG_LHS_BP_P_PID_OP"],'2025-01-01 00:00:00','2025-01-01 23:45:00')

In [26]:
df

,time,KAWAI_U2_2ND_STG_LHS_BP_P_PID_OP
0,2025-01-01 00:00:00,502.749
1,2025-01-01 00:00:15,503.282
2,2025-01-01 00:00:30,503.749
3,2025-01-01 00:00:45,503.749
4,2025-01-01 00:01:00,503.749
...,...,...
5696,2025-01-01 23:44:00,508.379
5697,2025-01-01 23:44:15,508.379
5698,2025-01-01 23:44:30,508.379
5699,2025-01-01 23:44:45,508.379


In [4]:
data={"setpointPrimary": "KAWAI_U2_1ST_STG_LHS_MN_P_PID_SP",
"measureValuePrimary": "KAWAI_U2_1ST_STG_LHS_MN_P_PID_MEAS",
"outputPrimary"  : "KAWAI_U2_1ST_STG_LHS_MN_P_PID_OP",
"setpointSecondary": "KAWAI_U2_1ST_STG_LHS_MN_S_PID_SP",
"measureValueSecondary": "KAWAI_U2_1ST_STG_LHS_MN_S_PID_MEAS",
"controlvalveSecondary"  : "KAWAI_U2_1ST_STG_LHS_MN_S_PID_OP"}

{'setpointPrimary': 'KAWAI_U2_1ST_STG_LHS_MN_P_PID_SP',
 'measureValuePrimary': 'KAWAI_U2_1ST_STG_LHS_MN_P_PID_MEAS',
 'outputPrimary': 'KAWAI_U2_1ST_STG_LHS_MN_P_PID_OP',
 'setpointSecondary': 'KAWAI_U2_1ST_STG_LHS_MN_S_PID_SP',
 'measureValueSecondary': 'KAWAI_U2_1ST_STG_LHS_MN_S_PID_MEAS',
 'controlvalveSecondary': 'KAWAI_U2_1ST_STG_LHS_MN_S_PID_OP'}

In [ ]:
sensorTags=[v for k,v in data.items()]


In [27]:
df = getdata("sensordataKawai",sensorTags,'2025-01-01 00:00:00','2025-01-01 23:45:00')

In [28]:
for col in df.columns:
    if col != "time":
       fdf= {v:k for k,v in data.items() if k!= v}
       df= df.rename(columns=fdf)

In [23]:
fdf

{'KAWAI_U2_1ST_STG_LHS_MN_P_PID_SP': 'setpointPrimary',
 'KAWAI_U2_1ST_STG_LHS_MN_P_PID_MEAS': 'measureValuePrimary',
 'KAWAI_U2_1ST_STG_LHS_MN_P_PID_OP': 'outputPrimary',
 'KAWAI_U2_1ST_STG_LHS_MN_S_PID_SP': 'setpointSecondary',
 'KAWAI_U2_1ST_STG_LHS_MN_S_PID_MEAS': 'measureValueSecondary',
 'KAWAI_U2_1ST_STG_LHS_MN_S_PID_OP': 'controlvalveSecondary'}

In [29]:
df.head(5)

,time,measureValuePrimary,outputPrimary,setpointPrimary,measureValueSecondary,controlvalveSecondary,setpointSecondary
0,2025-01-01 00:00:00,489.831,500.0,508.0,421.74,0.11348,500.0
1,2025-01-01 00:00:15,489.831,500.0,508.0,421.74,0.11348,500.0
2,2025-01-01 00:00:30,489.831,500.0,508.0,421.74,0.11348,500.0
3,2025-01-01 00:00:45,489.831,500.0,508.0,421.74,0.11348,500.0
4,2025-01-01 00:01:00,489.831,500.0,508.0,421.74,0.11348,500.0
